In [ ]:
# i ran the last two big cells. The ones before were more testing phase.

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler

import os

In [3]:
def hypothesis_test_procedure(df1, df2, filename):   
    # Initialize an empty DataFrame to store p-values
    column_names = df1.columns
    p_values_list = []
    reject_list = []
    
    for col_name in column_names:
        
        df1_col = df1[col_name]
        df2_col = df2[col_name]
        
        # gesetz der großen zahlen, kann direkt einen parametrischen test nehmen

        # alpha
        alpha = 0.05
        reject_H0 = True
        
        statistic, p_value = stats.ttest_ind(df1_col, df2_col)

        # Check the p-value to determine statistical significance
        if p_value < alpha:
            reject_H0 = True     
        else:
            reject_H0 = False

            
        p_values_list.append(p_value)
        reject_list.append(reject_H0)
        
        
    p_value_df = pd.DataFrame([p_values_list], columns=column_names, index=['p-value'])
    reject_df = pd.DataFrame([reject_list], columns=column_names, index=['Reject_H0'])
    return p_value_df, reject_df


In [20]:
scaler = MinMaxScaler()

directory_path = 'C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\syncedfiles'


dataframes = []
dataframes_reject = []


for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):

        file_path = os.path.join(directory_path, filename)

        df = pd.read_csv(file_path,low_memory=False)
        
        substring = 'peers'
        df_1 = df[~df['HyperactiveBehaviourType'].str.contains(substring,na=False)]  
        substring = 'self'
        df_wout = df_1[~df_1['HyperactiveBehaviourType'].str.contains(substring, na=False)]

        grouped_df = df_wout.groupby('Hyperactive_Restless')
        group_1_exists = 1 in grouped_df.groups
        group_2_exists = 2 in grouped_df.groups
        if group_1_exists and not group_2_exists:
            df_hyper = grouped_df.get_group(1)
            df_hyper = pd.DataFrame(scaler.fit_transform(df_hyper.iloc[:, 4:13]), columns=df_hyper.iloc[:, 4:13].columns)
        elif group_2_exists and not group_1_exists:
            df_hyper = grouped_df.get_group(2)
            df_hyper = pd.DataFrame(scaler.fit_transform(df_hyper.iloc[:, 4:13]), columns=df_hyper.iloc[:, 4:13].columns)
        elif group_1_exists and group_2_exists :
            value1_df = grouped_df.get_group(1)
            value2_df = grouped_df.get_group(2)
            # Concatenate the two DataFrames
            df_hyper = pd.concat([value1_df, value2_df])
            df_hyper = pd.DataFrame(scaler.fit_transform(df_hyper.iloc[:, 4:13]), columns=df_hyper.iloc[:, 4:13].columns)
            
        
        group_0_exists = 0 in grouped_df.groups
        if group_0_exists:
            df_nonhyper = grouped_df.get_group(0)
            df_nonhyper = pd.DataFrame(scaler.fit_transform(df_nonhyper.iloc[:, 4:13]), columns=df_nonhyper.iloc[:, 4:13].columns)
        
        if not df_hyper.empty and not df_nonhyper.empty:
            result_pvalue, result_reject = hypothesis_test_procedure(df_hyper,df_nonhyper,filename)

        
        #df_new = [result_pvalue, result_reject]
        #resulting_df = pd.concat(df_new, axis=0)
        resulting_df = result_pvalue
        reject_df = result_reject
        

        file_components = filename.split('_')
        resulting_df['Participent'] = file_components[1]
        resulting_df['Watch name'] = file_components[2]
        resulting_df['Activity'] = file_components[3]
        
        reject_df['Participent'] = file_components[1]
        reject_df['Watch name'] = file_components[2]
        reject_df['Activity'] = file_components[3]
        
        
        dataframes.append(resulting_df)
        dataframes_reject.append(reject_df)
        
        
# concatenated DataFrame
resulting_df_final = pd.concat(dataframes, axis=0)
reject_df_final = pd.concat(dataframes_reject, axis=0)
resulting_df_final.head()

#resulting_df_final.to_csv('C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\hypothesis_table.csv', index=False)
#reject_df_final.to_csv('C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\reject_hypothesis_table.csv', index=False)

,Accel_X,Accel_Y,Accel_Z,Gyro_X,Gyro_Y,Gyro_Z,Magno_X,Magno_Y,Magno_Z,Participent,Watch name,Activity
p-value,0.000000e+00,0.000000e+00,0.000000e+00,1.190087e-245,2.548231e-73,2.273920e-253,0.000000e+00,2.374879e-10,6.325207e-02,P10,0SSW4WearOS01,A3.csv
p-value,0.000000e+00,0.000000e+00,8.879578e-219,0.000000e+00,0.000000e+00,0.000000e+00,4.500540e-68,5.536011e-132,7.802592e-166,P10,SSW4WearOS02,A3.csv
p-value,1.246006e-59,9.590328e-46,7.032286e-221,3.461486e-23,2.434832e-02,1.859962e-236,1.820842e-12,3.366431e-18,1.745445e-16,P11,SSW4WearOS01,A1.csv
p-value,0.000000e+00,5.078682e-185,0.000000e+00,0.000000e+00,1.397381e-53,0.000000e+00,5.453677e-126,2.120716e-21,2.126595e-94,P11,SSW4WearOS01,A2.csv
p-value,1.448766e-118,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.772962e-210,6.929259e-58,P11,SSW4WearOS01,A3.csv


In [11]:
v =(0<0.5)
print(v)

True


In [22]:
file_path = 'C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\analysis_table_blobs_of_segments_with_mov_average_freq.csv'

df = pd.read_csv(file_path)
df.head()
grouped_df = df.groupby('type')
df_hyper = grouped_df.get_group(1)
df_nonhyper = grouped_df.get_group(0)

df_mean_hyper = df_hyper.iloc[:,:-8]
df_mean_nonhyper = df_nonhyper.iloc[:,:-8]

#df_mean_hyper = df_hyper.iloc[:,1:-1]
#df_mean_nonhyper = df_nonhyper.iloc[:,1:-1]


column_names = df_mean_hyper.columns
p_values_list = []
reject_list = []

# significance level
alpha = 0.05


for col_name in column_names:
    df1_col = df_mean_hyper[col_name]
    df2_col = df_mean_nonhyper[col_name]

    # Perform the t-test
    statistic, p_value = stats.ttest_ind(df1_col, df2_col)

    # Determine if the null hypothesis is rejected based on the p-value
    #reject_H0 = p_value < alpha
    if p_value < alpha:
        reject_H0 = 'Reject H0'
    else:
        reject_H0 = 'Accept H0' 

    p_values_list.append(p_value)
    reject_list.append(reject_H0)

# DataFrames for p_value and rejection choice
p_value_df = pd.DataFrame([p_values_list], columns=column_names, index=['p-value'])
reject_df = pd.DataFrame([reject_list], columns=column_names, index=['Reject_H0'])


results_df = pd.concat([p_value_df, reject_df])
results_df.head()
#results_df.to_csv('C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\hypothesis_table_from_analysis_table_blobs_of_segments_with_mov_average_freq.csv', index=False)

In [23]:
results_df.head()

,Accel_X_mean,Accel_Y_mean,Accel_Z_mean,Gyro_X_mean,Gyro_Y_mean,Gyro_Z_mean,Magno_X_mean,Magno_Y_mean,Magno_Z_mean,Accel_Vecdis_mean,...,Accel_Z_AUC_fft,Gyro_X_AUC_fft,Gyro_Y_AUC_fft,Gyro_Z_AUC_fft,Magno_X_AUC_fft,Magno_Y_AUC_fft,Magno_Z_AUC_fft,Accel_AUC_fft,Gyro_AUC_fft,Magno_AUC_fft
p-value,0.111184,0.634122,0.396814,0.471923,0.125295,0.868114,0.45896,0.008438,0.000001,0.546661,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Reject_H0,Accept H0,Accept H0,Accept H0,Accept H0,Accept H0,Accept H0,Accept H0,Reject H0,Reject H0,Accept H0,...,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0


In [5]:
results_df.head()

,Accel_X_mean,Accel_Y_mean,Accel_Z_mean,Gyro_X_mean,Gyro_Y_mean,Gyro_Z_mean,Accel_X_std,Accel_Y_std,Accel_Z_std,Gyro_X_std,...,Accel_Z_kurtosis,Gyro_X_kurtosis,Gyro_Y_kurtosis,Gyro_Z_kurtosis,Accel_X_skewness,Accel_Y_skewness,Accel_Z_skewness,Gyro_X_skewness,Gyro_Y_skewness,Gyro_Z_skewness
p-value,0.0,0.0,0.0,0.0,0.0,0.0,0.090823,0.0021,0.0,0.0,...,0.000039,0.387964,0.015675,0.031629,0.009019,0.003943,0.000001,0.0,0.0,0.130821
Reject_H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,...,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0,Reject H0


In [17]:
import pandas as pd
from scipy import stats

file_path = 'C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\analysis_table_blobs_of_segments_with_mov_average_freq.csv'

df = pd.read_csv(file_path)


column_names = df_mean_hyper.columns
p_values_list = []
reject_list = []
activity_list = []

# signifcance level
alpha = 0.05

activity_column = 'Activity'
type_column = 'type'



p_values_list = []
reject_list = []
activity_list = []

for activity in df[activity_column].unique():
    data_type_0 = df[(df[activity_column] == activity) & (df[type_column] == 0)]
    data_type_1 = df[(df[activity_column] == activity) & (df[type_column] == 1)]

    # Perform t-test
    t_statistic, p_value = stats.ttest_ind(data_type_0[column_names], data_type_1[column_names])

    # Determine if the null hypothesis is rejected based on the p-value
    reject_H0 = p_value < alpha
    #if p_value < alpha:
     #   reject_H0 = 'Reject H0'
    #else:
     #   reject_h = 'Accept H0' 

    p_values_list.append(p_value)
    reject_list.append(reject_H0)
    activity_list.append(activity)

# DataFrames for p_value and rejection choice
p_value_df = pd.DataFrame(p_values_list, columns=column_names, index=[activity_list])
reject_df = pd.DataFrame(reject_list, columns=column_names, index=[activity_list])

# Reset  index of p_value_df
p_value_df.reset_index(inplace=True)
reject_df.reset_index(inplace=True)

activity_df = pd.DataFrame(activity_list, columns=['Activity'])

p_value_df['Result'] = 'p_value'
reject_df['Result'] = 'reject'


#result1_df = pd.concat([activity_df, p_value_df], axis=1)
#result2_df = pd.concat([activity_df, reject_df], axis=1)

#result_df = pd.concat([result1_df, result2_df])
result_df = pd.concat([p_value_df, reject_df])

result_df.head(8)
#result_df.to_csv('C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\hypothesis_table_MA_seperate_activities.csv', index=False)

In [18]:
result_df.head(8)

,level_0,Accel_X_mean,Accel_Y_mean,Accel_Z_mean,Gyro_X_mean,Gyro_Y_mean,Gyro_Z_mean,Magno_X_mean,Magno_Y_mean,Magno_Z_mean,...,Gyro_X_AUC_fft,Gyro_Y_AUC_fft,Gyro_Z_AUC_fft,Magno_X_AUC_fft,Magno_Y_AUC_fft,Magno_Z_AUC_fft,Accel_AUC_fft,Gyro_AUC_fft,Magno_AUC_fft,Result
0,A1,0.367687,0.577182,0.131808,0.268674,0.156417,0.160080,0.669591,0.857320,0.006271,...,0.000002,0.000037,0.000004,0.000029,0.000020,0.000005,0.000006,0.000027,5.872040e-07,p_value
1,A2,0.418707,0.247394,0.113880,0.914313,0.793582,0.499859,0.010282,0.151636,0.205472,...,0.194424,0.331746,0.331293,0.113279,0.300130,0.335286,0.155568,0.135034,4.626662e-02,p_value
2,A3,0.939536,0.671630,0.764436,0.899029,0.115669,0.539223,0.171844,0.066081,0.000026,...,0.006472,0.026875,0.038797,0.002216,0.010543,0.011046,0.016391,0.033301,3.001358e-04,p_value
3,A4,0.023012,0.837615,0.508929,0.574143,0.809121,0.558782,0.467497,0.152600,0.501659,...,0.000501,0.000132,0.000335,0.000050,0.000125,0.000270,0.000155,0.000871,2.449217e-07,p_value
0,A1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,reject
1,A2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,reject
2,A3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,reject
3,A4,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,reject
